# Extract a Cameo Enterprise Architect Model

**The old way:** Open Cameo on a workstation, navigate the package browser, manually read diagrams, requirements, and blocks — then export each one individually.

**The new way:** Upload a `.mdzip` to Istari, run extraction, get structured SysML/UML data and all diagrams automatically.

This notebook:
1. Navigates the Istari system and configuration hierarchy
2. Runs Cameo extraction on an `.mdzip` file
3. Views the extracted blocks, requirements, and diagrams
4. Snapshots the results so the team can review what was extracted

See [`example-output/`](example-output/) for pre-computed results.

In [ ]:
# Setup
import sys, json
from pathlib import Path

try:
    import istari_digital_client
except ImportError:
    !pip install istari-digital-client python-dotenv -q

repo_root = str(Path.cwd().parent.parent)
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from istari_client import get_client

client = get_client()
user = client.get_current_user()
print(f"Connected as: {user.display_name} ({user.email})")

In [ ]:
# Explore the system — navigate the Istari hierarchy
#
# This system tracks one Cameo Enterprise Architect model file.
#
# Istari links:
#   System: https://demo.istari.app/systems/b3450e9e-d350-4940-a557-495f7efc9242

SYSTEM_ID = "b3450e9e-d350-4940-a557-495f7efc9242"  # Example: Extract Cameo Model
CONFIG_ID = "a8902f2d-f9cd-4a60-90d7-5dda2e4e8baa"  # Baseline configuration

system = client.get_system(SYSTEM_ID)
print(f"System: {system.name}")
print(f"  {system.description}\n")

configs = client.list_system_configurations(SYSTEM_ID, page=1, size=50)
print(f"Configurations ({configs.total}):\n")

for config in configs.items:
    print(f"  {config.name}")
    print(f"    Config ID: {config.id}")

    tracked = client.list_tracked_files(config.id, page=1, size=50)
    print(f"    Tracked files ({tracked.total}):")
    for tf in tracked.items:
        mode = tf.specifier_type.value
        print(f"      {tf.file_id} ({mode})")

    snapshots = client.list_snapshots(configuration_id=config.id, page=1, size=10)
    print(f"    Snapshots ({snapshots.total}):")
    for snap in snapshots.items:
        tags = client.list_tags(snapshot_id=snap.id, page=1, size=10)
        tag_names = [t.tag for t in tags.items]
        tag_str = f"  [{', '.join(tag_names)}]" if tag_names else ""
        revs = client.list_snapshot_revisions(snap.id, page=1, size=50)
        print(f"      {snap.id[:8]}...{tag_str}  ({revs.total} file(s))")
        for r in revs.items:
            size_kb = r.size / 1024
            print(f"        - {r.name} ({size_kb:.1f} KB)")

In [ ]:
# Load the model — show file details and revision history

MODEL_ID = "1fbd8f81-c4f7-4e4f-9140-e5004cf0fcf8"  # Istari_UAVOne.mdzip

model = client.get_model(MODEL_ID)
name = model.display_name or model.file.revisions[0].name
print(f"Model: {name}")
print(f"  File ID:   {model.file.id}")
print(f"  Revisions: {len(model.file.revisions)}")
for rev in model.file.revisions:
    size_kb = rev.size / 1024
    ver = f"  ({rev.version_name})" if hasattr(rev, 'version_name') and rev.version_name else ""
    print(f"    {rev.name} — {size_kb:.1f} KB{ver}")
print(f"  Artifacts: {len(model.artifacts)}")
print(f"\nTracked in config: {CONFIG_ID[:8]}... (LATEST)")

In [ ]:
# Run Cameo extraction
#
# Sends the .mdzip file to an Istari agent running Cameo Enterprise Architect.
# The agent extracts: blocks JSON, requirements JSON, and all diagrams as PNG.

from time import sleep
from datetime import datetime
from istari_digital_client import JobStatusName

print("Submitting Cameo extraction job...")
job = client.add_job(
    model_id=MODEL_ID,
    function="@istari:extract",
    tool_name="dassault_cameo",
    tool_version="2024x Refresh2",
    operating_system="Windows Server 2022",
    parameters={},
)
print(f"Job: {job.id}")

while True:
    sleep(10)
    job = client.get_job(job.id)
    ts = datetime.now().strftime("%H:%M:%S")
    status = job.status.name.value
    print(f"  [{ts}] {status}")
    if job.status.name in {JobStatusName.COMPLETED, JobStatusName.FAILED}:
        break

if job.status.name == JobStatusName.COMPLETED:
    print("\nExtraction complete!")
else:
    print("\nExtraction failed!")

In [ ]:
# View extraction results
#
# The agent extracted: blocks.json, requirements.json, and 46 PNG diagrams.

from IPython.display import HTML, display

model = client.get_model(MODEL_ID)
print(f"Artifacts: {len(model.artifacts)}\n")

for a in model.artifacts:
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev:
        size_kb = rev.size / 1024
        ext = rev.name.split('.')[-1].lower() if '.' in rev.name else ''
        print(f"  {rev.name} ({size_kb:.1f} KB)")

        # Show JSON contents inline
        if ext == 'json':
            try:
                data = json.loads(a.read_text())
                print(f"    {json.dumps(data, indent=2)[:500]}")
                if len(json.dumps(data, indent=2)) > 500:
                    print("    ...")
            except Exception:
                pass
            print()

In [ ]:
# Snapshot: capture state after extraction

from istari_digital_client import NewSnapshot, NewSnapshotTag

print("Creating snapshot: post-extraction...")
snap_response = client.create_snapshot(CONFIG_ID, NewSnapshot())
snapshot = snap_response.actual_instance

if hasattr(snapshot, "id"):
    snap_id = snapshot.id
    print(f"  New snapshot: {snap_id[:8]}...")
else:
    snaps = client.list_snapshots(configuration_id=CONFIG_ID, page=1, size=1)
    snap_id = snaps.items[0].id
    print(f"  No changes — tagging existing snapshot: {snap_id[:8]}...")

client.create_tag(snap_id, NewSnapshotTag(tag="post-extraction"))
print("  Tagged: post-extraction")

revs = client.list_snapshot_revisions(snap_id, page=1, size=50)
print(f"  Files: {revs.total}")

In [ ]:
# View system state — all snapshots with tags, files, and sizes

print(f"System: {system.name}")
print(f"  {system.description}\n")

configs = client.list_system_configurations(SYSTEM_ID, page=1, size=50)
for config in configs.items:
    print(f"Configuration: {config.name}")
    print(f"  ID: {config.id}")

    tracked = client.list_tracked_files(config.id, page=1, size=50)
    print(f"  Tracked files: {tracked.total}")

    snapshots = client.list_snapshots(configuration_id=config.id, page=1, size=20)
    print(f"\n  Snapshots ({snapshots.total}):\n")

    for i, snap in enumerate(snapshots.items):
        tags = client.list_tags(snapshot_id=snap.id, page=1, size=10)
        tag_names = [t.tag for t in tags.items]
        tag_str = ", ".join(tag_names) if tag_names else "untagged"
        revs = client.list_snapshot_revisions(snap.id, page=1, size=50)

        print(f"    {i+1}. [{tag_str}]  ({revs.total} files)")
        for r in revs.items:
            size_kb = r.size / 1024
            if size_kb > 1024:
                print(f"       {r.name} ({size_kb/1024:.1f} MB)")
            else:
                print(f"       {r.name} ({size_kb:.1f} KB)")
        print()

## Summary

**What we did:**
1. Uploaded a Cameo `.mdzip` file to Istari
2. Ran automated extraction — blocks JSON, requirements JSON, and all diagrams as PNG
3. Viewed the results without ever opening Cameo
4. Snapshotted the extraction results for the team

### Extraction Output

The extraction produces **48 artifacts** in total:

| Artifact | Size | Description |
|----------|------|-------------|
| `blocks.json` | 5.3 MB | All SysML blocks with properties and relationships |
| `requirements.json` | 24.2 KB | All SysML requirements with text and trace links |
| 46 x `*.png` | varies | Every diagram in the model, rendered as PNG |

### Version History

| # | Snapshot Tag | Files | What happened |
|---|-------------|-------|---------------|
| 1 | `initial-upload` | 1 | Raw `.mdzip` uploaded |
| 2 | `post-extraction` | 49 | `.mdzip` + 48 extracted artifacts |

### What Istari Did

| Step | Inner Loop (Cameo) | Outer Loop (Istari) |
|------|-------------------|---------------------|
| Upload model | — | Stored `.mdzip`, tracked as LATEST |
| Extract data | Opened model in Cameo Enterprise Architect, parsed SysML packages, exported blocks and requirements as JSON, rendered all diagrams as PNG | Ran job on Windows Server 2022 agent, stored all 48 artifacts |
| Snapshot | — | Captured point-in-time state with all artifacts tagged `post-extraction` |

### What's Next?

- **Query requirements** — use `requirements.json` to programmatically search and filter SysML requirements
- **Inspect blocks** — traverse `blocks.json` to map system structure and property values
- **Check design** — cross-reference extracted requirements with design parameters from other tools
- **Compare revisions** — upload a modified `.mdzip`, re-extract, and diff the JSON outputs across versions